<a href="https://colab.research.google.com/github/mburkasov/Financial-forecasting/blob/main/Case_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib as plt
import pandas as pd
import statsmodels.api as sm

from math import sqrt
from google.colab import drive
from sklearn import preprocessing
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [19]:
drive.mount('/content/drive')
df1 = pd.read_excel('/content/drive/My Drive/Colab Notebooks/job_satisfaction_train.xlsx')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
df = df1.copy()
#замена текстовых значений
for column in df.select_dtypes(include='object'):
  if len(df[column].value_counts().tolist()) < 10:
    for n, title in enumerate(df[column].value_counts().index.tolist()):
      df[column] = df[column].replace(title, n)
#отсортировать по макс значению
df = df.loc[:, df.max().sort_values(ascending=False).index]
#исправить интернет 8 и 9
df['internet'][df['internet']>1] = 1
#переставить столбик
jobsat = df.pop('jobsat').dropna().astype('int64')
print(jobsat)
df.insert(0,'jobsat',jobsat)
#колонки 
predict = df.iloc[5400:,1:]
df = df.dropna()
#сделать категориями
#for column in df:
#  if df[column].max() < 10:
#    df[column] = df[column].astype('category')

0       5
1       4
2       3
3       1
4       2
       ..
5395    2
5396    2
5397    4
5398    3
5399    1
Name: jobsat, Length: 5400, dtype: int64


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [15]:
#функция вифа
def getvif(X):
  vif = pd.DataFrame()
  vif["variables"] = X.columns
  X = np.array(X, dtype = float)
  vif["VIF"] = [variance_inflation_factor(X, i) for i in range(X.shape[1])]
  vif = vif.sort_values(by=['VIF'], ascending=False)
  return vif
#prediction and export
def export_prediction(model, x, columns = x.columns, filename = 'output'):
  pred = model.predict(sm.add_constant(x[columns]))
  export=pd.concat([pd.DataFrame(pred),pd.DataFrame(x)],axis=1)
  export.to_excel(f'/content/drive/My Drive/Colab Notebooks/{filename}.xlsx')
#forward selection
def forward_selection(data, target):
    all_columns = data.columns.tolist()
    model_columns = []
    maxf = 0
    while (len(all_columns)>0):
      all_columns = list(set(all_columns)-set(model_columns))
      currmaxf = 0
      for column in all_columns:
        model = sm.OLS(target, sm.add_constant(data[model_columns+[column]])).fit()
        new_f = model.fvalue
        if (currmaxf < new_f):
          currmaxf = new_f
          best_column = column

      if (maxf < currmaxf):
        maxf = currmaxf

        model_columns.append(best_column)
      else:
        break
    return model_columns
#backwards elimination
def backward_elimination(data, target, crit = 0.05):
    all_columns = data.columns.tolist()
    bad_columns = []
    while (len(all_columns)>1):
      all_columns = list(set(all_columns)-set(bad_columns))
      model = sm.OLS(target, sm.add_constant(data[all_columns])).fit()
      pval = model.pvalues.sort_values(ascending=False)
      if pval[0]>crit:
        bad_columns.append(pval.index[0])
      else:
        break
    return all_columns
#standart error of the model
def model_error (data, columns, step = 600, type = 'OLS'):
  data = data[['jobsat']+columns].dropna()
  predict = []
  for i in range(0,len(data.index),step):
    train = data.iloc[np.r_[0:i, i+step:5400],:]
    pred = data.iloc[i:i+step,1:]
    y = train.iloc[:,0]
    x = train.iloc[:,1:]
    x = sm.add_constant(x)
    pred = sm.add_constant(pred)
    model = getattr(sm,type)(y,x).fit()
    predict += list(model.predict(pred))
  data['prediction'] = predict
  data['sterr'] = (data['jobsat'] - data['prediction']) ** 2
  sterr = sqrt(data['sterr'].sum() / (len(data.index) - 1 - len(columns)))
  print('St. err:', sterr)
  return(sterr)

In [37]:
X = df.iloc[0:5400,1:]
#X = sm.add_constant(X)
getvif(X)

,variables,VIF
21,owntv,69.866298
8,inccat,53.414123
23,owncd,42.600807
9,carcat,42.003258
10,empcat,32.734368
22,ownvcr,32.452232
2,age,31.785612
1,car,21.390237
3,employ,14.166332
14,response,9.443690


In [ ]:
df.corr().round(2)

In [38]:
#eliminating VIF more than 10
crit = 10
X = df.iloc[0:5400,1:]
vif = getvif(X)
poplist = []
while vif.iloc[0,1]>crit:
  poplist.append(vif.iloc[0,0])
  X.pop(vif.iloc[0,0])
  vif = getvif(X)
print('popped:',poplist)
df = df.drop(poplist, axis=1)

popped: ['owntv', 'inccat', 'owncd', 'empcat', 'age', 'carcat', 'ownvcr', 'car']


In [39]:
x = df.iloc[0:5400,1:]
y = df.iloc[0:5400,0]
col1 = forward_selection(x,y)
print(col1)
X = df[col1].iloc[0:5400,0:]
X = sm.add_constant(X)
Y = df.iloc[0:5400,0]
model = sm.OLS(Y,X).fit()
print(model.summary())
model_error(df, col1)

['employ']
                            OLS Regression Results                            
Dep. Variable:                 jobsat   R-squared:                       0.237
Model:                            OLS   Adj. R-squared:                  0.236
Method:                 Least Squares   F-statistic:                     1673.
Date:                Thu, 11 Nov 2021   Prob (F-statistic):          9.52e-319
Time:                        18:51:14   Log-Likelihood:                -8641.6
No. Observations:                5400   AIC:                         1.729e+04
Df Residuals:                    5398   BIC:                         1.730e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.3327      0.024     96.9

1.1995493701216402

In [40]:
type = 'GLSAR'
x = df.iloc[0:5400,1:]
y = df.iloc[0:5400,0]
col2 = backward_elimination(x,y)
print(col2)
X = df[col2].iloc[0:5400,0:]
X = sm.add_constant(X)
Y = df.iloc[0:5400,0]
model2 = getattr(sm,type)(Y,X).fit()
print(model2.summary())
model_error(df, col2, 600, type)
export_prediction(model2, predict, col2, 'finalpred')

['employ', 'address', 'retire', 'reside', 'marital']
                           GLSAR Regression Results                           
Dep. Variable:                 jobsat   R-squared:                       0.241
Model:                          GLSAR   Adj. R-squared:                  0.240
Method:                 Least Squares   F-statistic:                     342.0
Date:                Thu, 11 Nov 2021   Prob (F-statistic):          3.48e-319
Time:                        18:51:19   Log-Likelihood:                -8624.6
No. Observations:                5399   AIC:                         1.726e+04
Df Residuals:                    5393   BIC:                         1.730e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------

In [43]:
allcol = list(set(df.columns.tolist()) - {'jobsat'})
print(allcol)
X = df.iloc[0:5400,1:]
X = sm.add_constant(X)
Y = df.iloc[0:5400,0]
model = sm.OLS(Y,X).fit()
print(model.summary())
model_error(df, allcol)

['callwait', 'callid', 'multline', 'ed', 'gender', 'address', 'internet', 'ownpc', 'news', 'ownpda', 'response', 'voice', 'wireless', 'pager', 'employ', 'ownfax', 'income', 'retire', 'marital', 'reside']
                            OLS Regression Results                            
Dep. Variable:                 jobsat   R-squared:                       0.243
Model:                            OLS   Adj. R-squared:                  0.240
Method:                 Least Squares   F-statistic:                     86.23
Date:                Thu, 11 Nov 2021   Prob (F-statistic):          9.23e-306
Time:                        18:52:02   Log-Likelihood:                -8619.5
No. Observations:                5400   AIC:                         1.728e+04
Df Residuals:                    5379   BIC:                         1.742e+04
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
      

1.2014566376442997

In [ ]:
#bar charts
import seaborn as sns
dfgraph = df.select_dtypes([np.int, np.float])

for i, col in enumerate(dfgraph.columns):
  plt.pyplot.figure()
  sns.barplot(data = dfgraph, x = 'jobsat',  y = col)